# Construct AAS (template) from SHACL Shapes graph

In [1]:
from rdflib import BNode, ConjunctiveGraph, Dataset, Graph, URIRef
from pyshacl import validate

In [2]:
prefixes = {
    'prov': 'http://www.w3.org/ns/prov#',
    'mas4ai': 'http://example.org/MAS4AI_GenericModel#',
    'aas': 'https://admin-shell.io/aas/3/0/RC01/',
    'aasenv': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShellEnvironment/',
    'aasaas': 'https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShell/',
    'aassm': 'https://admin-shell.io/aas/3/0/RC01/Submodel/',
    'aassmc': 'https://admin-shell.io/aas/3/0/RC01/SubmodelElementCollection/',
    'aasrefer': 'https://admin-shell.io/aas/3/0/RC01/Referable/',
    'aasrel': 'https://admin-shell.io/aas/3/0/RC01/RelationshipElement/',
    'aasdata': 'https://admin-shell.io/aas/3/0/RC01/HasDataSpecification/',
    'aasprop': 'https://admin-shell.io/aas/3/0/RC01/Property/',
    'aasrange': 'https://admin-shell.io/aas/3/0/RC01/Range/',
    'aassem': 'https://admin-shell.io/aas/3/0/RC01/HasSemantics/',
    'aasref': 'https://admin-shell.io/aas/3/0/RC01/Reference/',
    'aaskey': 'https://admin-shell.io/aas/3/0/RC01/Key/',
    'aasida': 'https://admin-shell.io/aas/3/0/RC01/Identifiable/',
    'aaside': 'https://admin-shell.io/aas/3/0/RC01/Identifier/',
    'aaskeyt': 'https://admin-shell.io/aas/3/0/RC01/KeyType/',
    'aaskind': 'https://admin-shell.io/aas/3/0/RC01/HasKind/',
    'aasmod': 'https://admin-shell.io/aas/3/0/RC01/ModelingKind/'
}

def add_prefixes(graph):
    for k,v in prefixes.items():
        graph.namespace_manager.bind(k, URIRef(v))

Questions:
* How to determine what resource to model as what type of AAS element?
* How to determine what should be modelled as an aas:ReferenceElement vs an aas:SubmodelCollection?
    * If more than one relation of the same type? -> allows to construct for example 'Jobs' SubmodelCollection, but not to model coordinates as a submodel collection.

Assumptions/modelling choices:
* Input should include definition of what `sh:NodeShape` to map to `aas:AssetAdministrationShell` (and `aas:Submodel`?);
* Use semantic ids to relate all AAS components together (so first construct all basic components and add relations afterwards)?

## Initialize data model/graph

In [13]:
dataset = Dataset()

g_sh = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/shapesGraph'))
g_sh.parse('examples/Example_ServoDCMotor.shape.ttl')

g_AAS_ont = dataset.graph(identifier=URIRef('https://admin-shell.io/aas/3/0/RC01/'))
g_AAS_ont.parse('https://raw.githubusercontent.com/admin-shell-io/aas-specs/master/schemas/rdf/rdf-ontology.ttl', format='text/turtle')

g_AAS = dataset.graph(identifier=URIRef('http://mas4ai.eu/id/graph/aas'))

g_conj = ConjunctiveGraph(dataset.store)

aas_classes = [
    'http://example.org/ServoDCMotor'
]

for c in aas_classes:
    g_sh.add((URIRef(c), URIRef('http://example.org/MAS4AI_GenericModel#hasInterface'), BNode()))

## Construct AAS components
* Construct components, starting from the lowest level.
* Add (temporary) provenance data (`prov:wasDerivedFrom`) that can be used to link the different AAS components.

### Property
* The `sh:path` of a `sh:PropertyShape` should be mapped to the `aassem:semanticId` reference;
* A `sh:PropertyShape` without a `sh:class` is an `aas:Property`.

In [14]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Property
  ?Property_iri a aas:Property ;
    aasprop:valueType ?dataType ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:key [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?Property ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  ?PropertyShape a sh:PropertyShape ;
    rdfs:label ?idShort ;
    sh:path ?Property ;
    sh:datatype ?dataType ;
  .
  OPTIONAL { ?PropertyShape rdfs:comment ?description }

  FILTER NOT EXISTS { ?PropertyShape sh:class ?Class } #filter out reference properties

  BIND(iri(concat( "http://mas4ai.eu/id/property/", strafter(str(uuid()), "urn:uuid:") )) as ?Property_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

### ReferenceElement
* A `sh:PropertyShape` that has a `sh:class`, is mapped to a `aas:ReferenceElement`.

In [15]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
BASE <http://mas4ai.eu/id/WP4/>

CONSTRUCT {
  # Reference Element
  ?ReferenceElement_iri a aas:ReferenceElement ;
    aasprop:valueType ?dataType ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:key [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?ReferenceElement ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  ?PropertyShape a sh:PropertyShape ;
    rdfs:label ?idShort ;
    sh:path ?ReferenceElement ;
    sh:class ?Class ; #filter on reference properties
  .

  FILTER EXISTS {?Class mas4ai:hasInterface []} #otherwise it should be a SMC

  BIND(iri(concat( "http://mas4ai.eu/id/referenceElement/", strafter(str(uuid()), "urn:uuid:") )) as ?ReferenceElement_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

### Submodel Element Collection
* A `sh:PropertyShape` with `sh:maxCount > 1` should be embedded in an `aas:SubmodelElementCollection` (for example jobs) ('PropertyCollection');
* A `sh:NodeShape` of which the related `sh:targetClass` does not have `mas4ai:hasInterface` should be mapped to a `aas:SubmodelElementCollection` (Collection of Properties).

#### PropertyCollection

In [16]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
    aaskind:kind aasmod:TEMPLATE ;
    aasrefer:idShort ?SMCidShort  ;
    prov:wasDerivedFrom ?PropertyShape ;
  .
}
WHERE {
  {
    ?PropertyShape a sh:PropertyShape ;
      rdfs:label ?propertyLabel ;
      sh:path ?Property ;
      sh:maxCount ?maxCount ;
    .
    FILTER( ?maxCount > 1 )
  } UNION {
    ?PropertyShape a sh:PropertyShape ;
      rdfs:label ?propertyLabel ;
      sh:path ?Property ;
    .
    FILTER NOT EXISTS { ?PropertyShape sh:maxCount [] } #if sh:maxCount>1 or not defined then embed the property in an ElementCollection
  }

  BIND(iri(concat( "http://mas4ai.eu/id/smc/", strafter(str(uuid()), "urn:uuid:") )) as ?SMC_iri)
  BIND(concat(?propertyLabel, 's') as ?SMC_idShort)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

#### CollectionProperty

In [17]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel Element Collection (SMC)
  ?SMC_iri a aas:SubmodelElementCollection ;
#    aassem:semanticId [
#      a aas:Reference ;
#      aasref:key [
#        a aas:Key ;
#        aaskey:idType aaskeyt:IRI ;
#        aaskey:value ?Property ;
#      ] ;
#    ] ;
    prov:wasDerivedFrom ?NodeShape ;
  .
}
WHERE {
  ?NodeShape a sh:NodeShape ;
    rdfs:label ?idShort ;
    sh:targetClass ?Class ;
  .

  FILTER EXISTS {?NodeShape sh:property []}
  FILTER NOT EXISTS {?Class mas4ai:hasInterface []}

  BIND(iri(concat( "http://mas4ai.eu/id/smc/", strafter(str(uuid()), "urn:uuid:") )) as ?SMC_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relation between SMCs and properties + reference elements

In [18]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?SMC aassmc:value ?Value .
  }
}
WHERE {
  {
    ?SMC a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom/sh:property ?PropertyShape .

    VALUES ?ValueType {aas:Property aas:ReferenceElement}
    ?Value a ?ValueType ;
      prov:wasDerivedFrom ?PropertyShape .
  } UNION {
    ?SMC a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?PropertyShape .
    ?PropertyShape a sh:PropertyShape .
    ?Value prov:wasDerivedFrom ?PropertyShape .
  }
}
''')

### Submodel
* A `sh:PropertyGroup` is converted to an `aas:Submodel` and embeds all resources of type `sh:PropertyShape` that are linked to the `sh:PropertyGroup` via `sh:group`.

In [19]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Submodel (SM)
  ?SM_iri a aas:Submodel ;
    prov:wasDerivedFrom ?PropertyGroup, ?PropertyShape ;
  .

#    aasida:identification [
#      a aas:Identifier ;
#      aaside:idType aaskeyt:IRI ;
#      aaside:identifier ?SM ;
#    ]

}
WHERE {
  {
    ?PropertyGroup a sh:PropertyGroup ;
      rdfs:label ?idShort ;
    .
  } UNION {
    ?PropertyShape a sh:PropertyShape ;
      rdfs:label ?idShort ;
      ^sh:property/sh:targetClass/mas4ai:hasInterface [] ; # only properties that are directly related to a class that requires an AAS
    .
    FILTER NOT EXISTS {?PropertyShape sh:group []}
  }

    BIND(iri(concat( "http://mas4ai.eu/id/sm/", strafter(str(uuid()), "urn:uuid:") )) as ?SM_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relation between submodels and properties + submodel element collections

In [20]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Submodel aassm:submodelElement ?SubmodelElement .
  }
}
WHERE {
  {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom/^sh:group ?PropertyShape .

    ?SubmodelElement a aas:Property ;
      prov:wasDerivedFrom ?PropertyShape .
  } UNION {
    ?Submodel a aas:Submodel ;
      prov:wasDerivedFrom/^sh:group? ?PropertyShape .

    ?PropertyShape a sh:PropertyShape ;
      sh:class/^sh:targetClass ?NodeShape .

    ?SubmodelElement a aas:SubmodelElementCollection ;
      prov:wasDerivedFrom ?NodeShape .
  }
}
''')

### AssetAdministrationShell

In [21]:
add_prefixes(dataset)

g_AAS.parse(data=g_sh.query('''
CONSTRUCT {
  # Asset Administration Shell (AAS)
  ?AAS_iri a aas:AssetAdministrationShell ;
    aassem:semanticId [
      a aas:Reference ;
      aasref:key [
        a aas:Key ;
        aaskey:idType aaskeyt:IRI ;
        aaskey:value ?AASClass ;
      ] ;
    ] ;
    prov:wasDerivedFrom ?NodeShape ;
  .
}
WHERE {
  ?NodeShape a sh:NodeShape ;
    rdfs:label ?idShort ;
    sh:targetClass ?AASClass ;
  .

  ?AASClass mas4ai:hasInterface [] .

  BIND(iri(concat(str(?AASClass), '_', 'aas')) as ?AAS) #temporarily
  BIND(iri(concat( "http://mas4ai.eu/id/aas/", strafter(str(uuid()), "urn:uuid:") )) as ?AAS_iri)
}
''').graph.serialize())

<Graph identifier=http://mas4ai.eu/id/graph/aas (<class 'rdflib.graph.Graph'>)>

##### Relations between asset administration shell and submodels

In [22]:
add_prefixes(dataset)

g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?AAS aasaas:submodel ?Submodel .
  }
}
WHERE {
  ?AAS a aas:AssetAdministrationShell ;
    prov:wasDerivedFrom ?NodeShape .

  ?NodeShape a sh:NodeShape ;
    sh:property/sh:group? ?ShapeGroup .

  ?Submodel a aas:Submodel ;
    prov:wasDerivedFrom ?ShapeGroup .
}
''')

### AssetAdministrationShellEnvironment

## Add common statements for objects of a certain type
(if they don't exist yet)

### HasKind
`aaskind:kind`

In [23]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aaskind:kind aasmod:Template .
  }
}
WHERE {
  ?Object a/rdfs:subClassOf* aas:HasKind .
  FILTER NOT EXISTS { ?Object aaskind:kind [] }
}
''')

### Referable
|Shapes predicate|AAS predicate|
|---|---|
| `rdfs:label` | `aasrefer:idShort`     |
| `rdfs:comment` | `aasrefer:description` |
| `skos:prefLabel` | `aasrefer:displayName` |

In [24]:
g_conj.update('''
INSERT {
  GRAPH <http://mas4ai.eu/id/graph/aas> {
    ?Object aasrefer:idShort ?idShort ;
      aasrefer:description ?description ;
      aasrefer:displayName ?displayName ;
    .
  }
}
WHERE {
  ?Object a/rdfs:subClassOf* aas:Referable ;
    prov:wasDerivedFrom ?Shape .

  ?Shape rdfs:label ?shapeLabel .
  OPTIONAL { ?Object aasrefer:idShort ?_idShort }
  BIND ( COALESCE(?_idShort, ?shapeLabel) AS ?idShort )

  OPTIONAL {
    ?Shape rdfs:comment ?shapeComment .
    OPTIONAL { ?Object aasrefer:description ?_description }
    BIND ( COALESCE(?_description, ?shapeComment) AS ?description )
  }

  OPTIONAL {
    ?Shape skos:prefLabel ?shapePrefLabel .
    OPTIONAL { ?Object aasrefer:displayName ?_displayName }
    BIND ( COALESCE(?_displayName, ?shapePrefLabel) AS ?displayName )
  }
}
''')

## Inspect AAS graph

In [25]:
print(g_AAS.serialize())

@prefix aas: <https://admin-shell.io/aas/3/0/RC01/> .
@prefix aasaas: <https://admin-shell.io/aas/3/0/RC01/AssetAdministrationShell/> .
@prefix aaskey: <https://admin-shell.io/aas/3/0/RC01/Key/> .
@prefix aaskeyt: <https://admin-shell.io/aas/3/0/RC01/KeyType/> .
@prefix aaskind: <https://admin-shell.io/aas/3/0/RC01/HasKind/> .
@prefix aasmod: <https://admin-shell.io/aas/3/0/RC01/ModelingKind/> .
@prefix aasprop: <https://admin-shell.io/aas/3/0/RC01/Property/> .
@prefix aasref: <https://admin-shell.io/aas/3/0/RC01/Reference/> .
@prefix aasrefer: <https://admin-shell.io/aas/3/0/RC01/Referable/> .
@prefix aassem: <https://admin-shell.io/aas/3/0/RC01/HasSemantics/> .
@prefix aassm: <https://admin-shell.io/aas/3/0/RC01/Submodel/> .
@prefix aassmc: <https://admin-shell.io/aas/3/0/RC01/SubmodelElementCollection/> .
@prefix ex: <http://example.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/X

In [ ]:
# Remove provenance statements
g_AAS.update('''
DELETE {
  ?s prov:wasDerivedFrom ?o
}
WHERE {
  ?s prov:wasDerivedFrom ?o
}
''')

# Store to file
g_AAS.serialize('examples/Example_ServoDCMotor.aas.ttl')
# g_AAS.serialize('examples/Example_ServoDCMotor.aas.jsonld', format='json-ld')